# Generacion de imagen


In [1]:
import cv2
import numpy as np
import sys
import os.path
import glob
import torch
import architecture as arch

In [2]:
bImg = cv2.imread("arbol3.jpg")
overlayImage_tortuga = cv2.imread("tortuga_png1.png" , cv2.IMREAD_UNCHANGED)
overlayImage_conejo = cv2.imread("liebre1.png" , cv2.IMREAD_UNCHANGED)

# Mantener el aspecto de la tasa de la imagen original
Cambia el tamaño sin distorsion

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [4]:
image_arbol = image_resize(bImg, height = 200)
overlayImage_tortuga = image_resize(overlayImage_tortuga, height = 40)
overlayImage_conejo = image_resize(overlayImage_conejo, height = 40)

In [5]:

def transparentOverlay(src , overlay , pos=(0,0),scale = 1):

    overlay = cv2.resize(overlay,(0,0),fx=scale,fy=scale)

    h,w,_ = overlay.shape  # Size of foreground

    rows,cols,_ = src.shape  # Size of background Image

    y,x = pos[0],pos[1]    # Position of foreground/overlay image


    for i in range(h):

        for j in range(w):

            if x+i >= rows or y+j >= cols:

                continue

            alpha = float(overlay[i][j][3]/255.0) # read the alpha channel 

            src[x+i][y+j] = alpha*overlay[i][j][:3]+(1-alpha)*src[x+i][y+j]

    return src


In [6]:
result = transparentOverlay(image_arbol,overlayImage_tortuga,(55,155),1)
result2 = transparentOverlay(result,overlayImage_conejo,(130,145),1)

# RSGAN

In [7]:
model_path ='./models/RRDB_ESRGAN_x4.pth'#sys.argv[1]  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
#device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
device = torch.device('cpu')

test_img_folder = 'LR/*'

model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=4, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)
img = result2 * 1.0 / 255
img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
img_LR = img.unsqueeze(0)
img_LR = img_LR.to(device)

output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
output = (output * 255.0).round()
cv2.imwrite('rsgan.png',output)

C:\Users\Gonzalo\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


True

# Ecualización de Histograma en Python
Mejora el contraste de la imagen

In [22]:
img = cv2.imread('rsgan.png')

In [23]:
img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)

In [24]:
img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])

In [25]:
hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)

# Filtrado de imagenes

In [41]:
blur_equ = cv2.GaussianBlur(hist_equalization_result,(3,3),0)

# Resize de la imagen resultante

In [42]:
image_final = image_resize(blur_equ, height = 350)
cv2.imwrite('Collage_animalibro_final.jpg',image_final)

True